In [1]:
# Add directory above current directory to path
import sys as SYS; SYS.path.insert(0, '..')

# for saving
import os
import csv

from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

from src import setup
# setup.use_gpu()

In [2]:
from src import ( 
    measurements as measure,
    density_matrix as DM,
    simulation as sim,
    orders,
    order_rules,
    random_unitary,
    simulation)

from Scripts import simulation_CLI as cleo

In [3]:
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors

import pandas as pd
from scipy.stats import skew, kurtosis

import matplotlib.animation as animation
# Add directory above current directory to path
import sys as SYS; SYS.path.insert(0, '../..')
import h5py
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from src import ket as ket
from src import density_matrix as DM

In [4]:
# This initializes the file. Don't need to do it after the first time
import csv
with open ('RunStats.csv', mode='w') as csv_file:
    fieldnames = ['runtype_name', 'tz_mean','tz_std','MI_mean', 'MI_std', 'disp_mean','disp_std','clust_mean','clust_std', 'DeltaW_mean', 'DeltaW_std','DeltaW_persist_stat','DeltaW_persist_std']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()

In [ ]:
# Specify the type of initial conditions and the number of qubits

In [191]:
dataset_label = "pure" #should come from this set {pure,therm,inhomov2,therm}
num=12 # the number of qubits in the network
num_seeds=100 # the number of seeds
steps=499 # the number of steps in any one evolution
denom = 15 # the angle denominator chosen for Haar Q unitary eg pi/15 then denom =15

In [ ]:
# Assuming the large .hdf5 files have already been generated. If not, us any of the Data_Analysis_Big_Landsape notebooks to create them

In [192]:
newdata5 = dict(h5py.File("../data/unnati_submit_12_pure_strongmax/unnati_submit_12_pure_strongmax.hdf5"))
newdata2 = dict(h5py.File("../data/unnati_submit_12_pure_greedy/unnati_submit_12_pure_greedy.hdf5"))
newdata3 = dict(h5py.File("../data/unnati_submit_12_pure_mimic/unnati_submit_12_pure_mimic.hdf5"))
newdata4 = dict(h5py.File("../data/unnati_submit_12_pure_landmax/unnati_submit_12_pure_landmax.hdf5"))
newdata1 = dict(h5py.File("../data/unnati_submit_12_pure_random/unnati_submit_12_pure_random.hdf5"))

In [6]:
newdata5 = dict(h5py.File("../data/unnati_submit_12_therm_strong_max/unnati_submit_12_therm_strong_max.hdf5"))
newdata2 = dict(h5py.File("../data/unnati_submit_12_therm_greedy/unnati_submit_12_therm_greedy.hdf5"))
newdata3 = dict(h5py.File("../data/unnati_submit_12_therm_mimic/unnati_submit_12_therm_mimic.hdf5"))
newdata4 = dict(h5py.File("../data/unnati_submit_12_therm_landmax/unnati_submit_12_therm_landmax.hdf5"))
newdata1 = dict(h5py.File("../data/unnati_submit_12_therm_random/unnati_submit_12_therm_random.hdf5"))

In [84]:
newdata5 = dict(h5py.File("../data/unnati_submit_12_inhomo_v1_strong_max/unnati_submit_12_inhomo_v1_strong_max.hdf5"))
newdata2 = dict(h5py.File("../data/unnati_submit_12_inhomo_v1_greedy/unnati_submit_12_inhomo_v1_greedy.hdf5"))
newdata3 = dict(h5py.File("../data/unnati_submit_12_inhomo_v1_mimic/unnati_submit_12_inhomo_v1_mimic.hdf5"))
newdata4 = dict(h5py.File("../data/unnati_submit_12_inhomo_v1_landmax/unnati_submit_12_inhomo_v1_landmax.hdf5"))
newdata1 = dict(h5py.File("../data/unnati_submit_12_inhomo_v1_random/unnati_submit_12_inhomo_v1_random.hdf5"))

In [72]:
newdata1 = h5py.File("../data/random/random.hdf5")
newdata2 = h5py.File("../data/greedy/greedy.hdf5")
newdata3 = h5py.File("../data/mimic/mimic.hdf5")
newdata4 = h5py.File("../data/landscape_maximizes/landscape_maximizes.hdf5")
newdata5 = h5py.File("../data/strongest_maximizes/strongest_maximizes.hdf5")
# Initialize the file to write all results to

In [ ]:
#The physics functions needed

In [132]:
def get_pops(data, n_qubits, connectivity,update_rule):
    #dimension 0 is each trial
    #dimension 1 is each time step
    #dimension 2 is each qubit
    result = []
    for trial in data[f'{n_qubits} qubits'][f'{connectivity} connectivity']['unitary energy subspace 1']:
        seed = trial.split(' ')[-1]
        dat = dict(data[f'{n_qubits} qubits'][f'{connectivity} connectivity']['unitary energy subspace 1'][f'unitary seed {seed}'][f'ordering seed {update_rule}']['pops'])
        dat = {int(k.split('(')[0]): dat[k] for k in dat}
        dat = np.array([np.array([dat[k][subkey][()] for subkey in sorted(dat[k])]) for k in sorted(dat)])
        result.append(dat)
    return(np.array(result))

def get_2_qbit_dms(data, n_qubits, connectivity,update_rule):
    basis = ket.canonical_basis(2)
    #dimension 0 is each trial
    #dimension 1 is each time step (recall that sampling step is 5 by default)
    #dimension 2 is qubit pair
    #dimension 3 and 4 is the 2 qubit density matrix
    result = []
    
    def to_tuple(string):
        tuple_elements = string.strip('()').split(',')
        return tuple(int(elem.strip()) for elem in tuple_elements)

    for trial in data[f'{n_qubits} qubits'][f'{connectivity} connectivity']['unitary energy subspace 1']:
        seed = trial.split(' ')[-1]
        dat = dict(data[f'{n_qubits} qubits'][f'{connectivity} connectivity']['unitary energy subspace 1'][f'unitary seed {seed}'][f'ordering seed {update_rule}']['two_qubit_dms'])
        dat = {int(k.split('(')[0]): dat[k] for k in dat}
        
        dat = np.array([{to_tuple(subkey):DM.DensityMatrix(dat[k][subkey],basis) for subkey in sorted(dat[k])} for k in sorted(dat)])
        result.append(dat)
    return(np.array(result))

In [133]:
def extractable_work_of_one_trial(pops,trial):
    return np.array([measure.extractable_work_of_each_qubit_from_pops(p) for p in pops[trial]])

In [134]:
def extractable_work_of_all_trials(pops):
    ext_work_all = []
    for trial in range(100):
        ext_work_all.append(extractable_work_of_one_trial(pops,trial))
    return ext_work_all

In [135]:
def change_in_ext_work(ext_work_one_trial):
    return np.diff(ext_work_one_trial,axis = 0)

def change_in_ext_work_all_trials(ext_work_all_trial):
    change_in_ext_all =[]
    for trial in range(100):
        change_in_ext_all.append(np.diff(ext_work_all_trial[trial],axis = 0))
    return change_in_ext_all

In [136]:
#MI of every pair
def mutual_info_dicts(twoQdms, trial_index):
    mutual_info_list = []
    for time_step in twoQdms[trial_index]:
        mutual_info_dict = {}
        mutual_info = measure.mutual_information_of_every_pair_dict(time_step)
        # Filter out values below the precision threshold
        filtered_mutual_info = {k: v if v >= 1e-6 else 0 for k, v in mutual_info.items()}
        # Update the mutual_info_dict with the filtered mutual info for the current time step
        mutual_info_dict.update(filtered_mutual_info)
        # Append the mutual_info_dict to the list
        mutual_info_list.append(mutual_info_dict)
    return mutual_info_list

In [137]:
# Call the two_point_dict function for each trial index
# Get the list of dictionaries containing two point values for each time step
def create_adjacency_matrix_two_dim(two_point_dict, num_nodes):
    adjacency_matrix = [[0] * num_nodes for _ in range(num_nodes)]
    for (node1, node2), two_point_value in two_point_dict.items():
        adjacency_matrix[node1][node2] = two_point_value
        adjacency_matrix[node2][node1] = two_point_value  # Assuming undirected graph
    return adjacency_matrix

In [138]:
#create adjacency matrix

In [139]:
def adjacency_matrices_list_two_dim(twoQdms, trial_index, num_nodes, two_point_dicts):
    # Get the list of dictionaries containing mutual information values for each time step
    two_point_dicts_list = two_point_dicts(twoQdms, trial_index)
    # Create adjacency matrices for each time step
    adjacency_matrices = []
    for two_point_dict in two_point_dicts_list:
        adj_matrix = create_adjacency_matrix_two_dim(two_point_dict, num_nodes)
        adjacency_matrices.append(adj_matrix)
    return adjacency_matrices

In [140]:
#Generates a giant list of adjacency matrices for each seed. Is a list of lists of 500 adjacency matrices each

In [141]:
def generate_all_adjacency_matrices(twoQdms, num_seeds, num_qubits, mutual_info_dicts):
    # Initialize an empty list to store the adjacency matrices for each trial
    all_trials_adjacency_matrices = []
    
    # Loop over each seed from 0 to num_seeds-1
    for seed in range(num_seeds):
        # Generate an empty list to store adjacency matrices for this trial
        trial_matrices = []
        
        # Generate the adjacency matrix for the given seed
        adj_matrix = adjacency_matrices_list_two_dim(twoQdms, seed, num_qubits, mutual_info_dicts)
        
        # Append each generated adjacency matrix to the trial-specific list
        trial_matrices.append(adj_matrix)
        
        # Append the list of matrices for this trial to the overall list
        all_trials_adjacency_matrices.append(trial_matrices)
    
    # Convert to a NumPy array if needed
    all_trials_adjacency_matrices_array = np.array(all_trials_adjacency_matrices)
    
    return all_trials_adjacency_matrices_array


In [142]:
#Total Mutual information on the network. This serves as a measure compared to the maximum mutual information that can develop on network
def MItotal(MI_adj):
    MI_adj_tot=[]
    for i in range(len(MI_adj)):
       MI_adj_tot.append(np.sum(MI_adj[i],axis=0))
    return np.array(MI_adj_tot)

In [143]:
#Clustering and disparity of network as a function of time: Can run with ensemble average set of adjacency matrix

In [144]:
#Clustering Coefficient
def clustering_coeff(adjacency_matrix_list):
    C_list = []
    #adjacency_matrix_list=np.array(adjacency_matrix_list)
    for adj_mat in adjacency_matrix_list:
        adj_mat=np.array(adj_mat)
        M_sq = adj_mat@adj_mat
        sum_of_M_sq = np.sum(M_sq)
        M_cube = adj_mat@adj_mat@adj_mat
        M_cube_trace = np.trace(M_cube)
        C_list.append(M_cube_trace/sum_of_M_sq)
    return np.array(C_list)

#Disparity
def disparity_function(adjacency_matrix_list, N):
    D_list = []
    for adj_mat in adjacency_matrix_list:
        M_row_sum_squared = np.sum(adj_mat, axis=1)**2
        M_row_sum_of_squared_elements = np.sum(adj_mat**2, axis=1)
        # Check for zero division before performing division
        Di = np.where(M_row_sum_squared != 0, M_row_sum_of_squared_elements / M_row_sum_squared, 0)
        D_list.append(np.sum(Di) / N)
    return np.array(D_list)

In [17]:
# find the the numerical value of shift per qubit map in one trial index i.e. one full time series of evolution

def tz_two_q_dyns_map(pops_trial_index,denom): 
    tz = []
    np.array(pops_trial_index)
    for step_index in range(steps):
        tzi =  np.sin(np.pi/denom)*np.sin(np.pi/denom) -2*(pops_trial_index[step_index + 1 ] - np.cos(np.pi / denom)*np.cos(np.pi / denom)* pops_trial_index[step_index])
        tz.append(tzi)
    return tz

# Now put together the above to make the full list for all seeds
    #dimension 0 is each trial
    #dimension 1 is each time step
    #dimension 2 is each qubit
def tz_full_array(pops_all_trials):
    tz_full_list=[]

    for s in range(99):       
        tz_full_list.append(tz_two_q_dyns_map(pops_all_trials[s],denom))

    return np.array(tz_full_list)
    

In [27]:
# The averaging procedures needed

In [18]:
#Returns ensembled average one point dataset ie data should look like a full seed
def late_time_average_of_list(list_one_trial):
    late_time_data=np.delete(list_one_trial,slice(0,201),1)
    late_time_data=np.delete(list_one_trial,slice(51,301),1)
    ensemble_average = np.mean(late_time_data, axis=0)  # Shape (100, N)
    return (ensemble_average)

In [19]:
#Performs late time avergaing (200-250 steps), then ensemble averages it for all trials, then takes the mean and std between qubits

In [20]:
#Use for DeltaWex and Tz
#Returns late time value of one point measures for all qubits and then the mean and standard deviation between qubits
def late_time_averages_one_point_all_seeds_between_q(datasets):
    # Trim down the arrays to a segment of late time data
    # Assuming datasets is a list of time steps (axis 0) for each qubit (axis 1), already averaged over seeds
    
        late_time_data=np.delete(datasets,slice(0,201),1)
        late_time_data=np.delete(late_time_data,slice(51,301),1)
    
        # Take the average over the late time values
        late_time_ave=np.mean(late_time_data,axis=1)
    
        # Take the average over seeds
        late_time_seed_ave=np.mean(late_time_ave,axis=0)    
        
        # Finally, define the mean for each qubit, and the std deviation of qubits
        qubit_mean=np.mean(late_time_seed_ave)
        stddev_over_qubits=np.std(late_time_seed_ave)
        return qubit_mean,stddev_over_qubits

In [21]:
#Returns late time value of two point matrix
def late_time_averages_two_point_one_seed(dataset):
    # Trim down the arrays to a segment of late time data
    # Assuming datasets is a list of time steps (axis 0) for each qubit (axis 1), already averaged over seeds
    late_time_data=np.delete(late_time_data,slice(0,201),0)
    late_time_data=np.delete(late_time_data,slice(51,301),0)

    # Take the average over the late time values
    late_time_ave=np.mean(late_time_data,axis=0)
    return late_time_ave

In [22]:
#Returns mean and std along each row, total along each row and std between total of each row
def stats_for_a_matix(matrix):
    mean_each_q=[]
    for row in matrix:
        mean_each_q.append(np.mean(row))
    std_each_q=[]
    for row in matrix:
        std_each_q.append(np.std(row))
    total_each_row=[]
    for row in matrix:
        total_each_q.append(np.total(row))
    std_between_total_each_row=np.std(total_each_row)
    return mean_each_q,std_each_q,total_each_q,std_between_total_each_row  

In [33]:
#Ensemble average across 100 data sets

In [34]:
#Time average until time t

In [23]:
def time_averaged_one_point_measures_at_t(dataset,t):
    # Assuming dataset is list of 100 lists i.e. one trial
    #stack datasets in 3D
    # Take the ensemble average along the first axis (averaging over all datasets)
    sum_till_t = np.sum(dataset[:t], axis=0)
    time_average_t = (1/(t+1))*(sum_till_t)
    return (time_average_t)

def time_averaged_one_point_measures_matrix_full_sim(dataset):
    # Take the ensemble average along the first axis (averaging over all datasets)
    time_average=[]
    for t in range(len(dataset)):
        time_average.append(time_averaged_one_point_measures_at_t(dataset,t))
    return (time_average)

In [24]:
#Returns ensembled average one point dataset ie data should look like a full seed
def ensemble_averaged_one_point_measures_mean_std(datasets):
    # Assuming datasets is a list of 100 datasets, where each dataset is a 2D array of shape (100, N)
    # Stack datasets into a 3D array
    stacked_data = np.stack(datasets)  # Shape (100, 100, N)

    # Take the ensemble average along the first axis (averaging over all datasets)
    ensemble_average = np.mean(stacked_data, axis=0)  # Shape (100, N)

    # Calculate the standard deviation along the first axis
    ensemble_std = np.std(stacked_data, axis=0) 
    return (ensemble_average, ensemble_std)

In [25]:
#Returns ensembled average matrix from the datasets ie data should look like data for a full seed
def ensemble_averaged_two_point_measures_mean(data):
    ensemble_avg_list = []
    # Loop through each time step
    for t in range(499):
        matrix_at_t_diff_trials=[]
        for seed in range(99):
            #matrix_at_t_diff_trials.append(data[seed][0][t])
            matrix_at_t_diff_trials.append(data[seed][t])
        # Calculate the average adjacency matrix for this time step
        avg_matrix_at_t = np.mean(matrix_at_t_diff_trials, axis=0) 
        # Append the averaged matrix to the ensemble list
        ensemble_avg_list.append(avg_matrix_at_t)
    return ensemble_avg_list

In [26]:
#Ensemble averaged list of values ed use for getting ensembled average clustering disparity etc:
def list_lists_of_values_measures(list_of_lists):
    #Assuming each list in side the list is data from some measure on the landscape for the entire seed of length 500
    #Example Clustering
    mean_list = np.mean(list_of_lists,axis=1)
    std_list = np.std(list_of_lists,axis=1)
    return (mean_list, std_list)

In [27]:
def ensemble_average_late_time_mean_std_value(list_of_lists):
    late_time_per_trial=[]
    for l in list_of_lists:
        late_time_per_trial.append(late_time_average_of_list(l))
    mean_value = np.mean(late_time_per_trial,axis=0)
    std_value = np.std(late_time_per_trial,axis=0)
    return (mean_value,std_value)

In [28]:
# Or, if the average over seeds has already been done

In [29]:
def late_time_averages_ensemble_ave(datasets):
    # Trim down the arrays to a segment of late time data
    # Assuming datasets is a list of time steps (axis 0) for each qubit (axis 1), already averaged over seeds
    late_time_data=np.delete(datasets,slice(0,201),0)
    late_time_data=np.delete(late_time_data,slice(51,301),0)

    # Take the average over the late time values
    late_time_ave=np.mean(late_time_data,axis=0)
    
    # Finally, define the mean for each qubit, and the std deviation of qubits
    qubit_mean=np.mean(late_time_ave)
    stddev_over_qubits=np.std(late_time_ave)
    return qubit_mean,stddev_over_qubits

In [30]:
# Use this to first define the adjacency matrices for each seed, where an average over late times has been done
# This can then be used to compute dispartiy, clustering for each seed, and mean and std dev over seeds
def late_time_averages_adj(datasets):
    late_time_avg_adj_list = []
    
    # Trim down the arrays to a segment of late time data
    # Assuming datasets is a list of 100 datasets (the seeds, axis 0), 
    # where each seed contains 500 steps (axis 1), and each step has a (num x num) (axis 2)
    late_time_data=np.delete(datasets,slice(0,201),2)
    late_time_data=np.delete(late_time_data,slice(51,301),2)

    # Take the average over the late time values
    late_time_ave=np.mean(late_time_data,axis=2)

     # Append the late-time-averaged matrix for each seed to the list of adjacency matrices
    for s in range(99):       
        late_time_avg_adj_list.append(late_time_ave[s][0])
    return late_time_avg_adj_list

In [43]:
# Now do the computations requred to compute the MI, disp, and clust statistics

In [44]:
# Finds the pops data for each rule, all the trials

In [193]:
num=12
pops_random_c2 = get_pops(newdata1, num, "c2_2local", "random")
pops_random_c4 = get_pops(newdata1, num, "c4_2local", "random")
#pops_random_c5 = get_pops(newdata1, num, "c5_2local", "random")
#pops_random_c6 = get_pops(newdata1, num, "c6_2local", "random")
#pops_random_cn = get_pops(newdata1, num, "cN_2local", "random")

In [194]:
pops_greedy_c2 = get_pops(newdata2, num, "c2_2local", "greedy")
pops_greedy_c4 = get_pops(newdata2, num, "c4_2local", "greedy")
#pops_greedy_c5 = get_pops(newdata2, num, "c5_2local", "greedy")
#pops_greedy_c6 = get_pops(newdata2, num, "c6_2local", "greedy")
#pops_greedy_cn = get_pops(newdata2, num, "cN_2local", "greedy")

In [195]:
pops_mimic_c2 = get_pops(newdata3, num, "c2_2local", "mimic")
pops_mimic_c4 = get_pops(newdata3, num, "c4_2local", "mimic")
#pops_mimic_c5 = get_pops(newdata3, num, "c5_2local", "mimic")
#pops_mimic_c6 = get_pops(newdata3, num, "c6_2local", "mimic")
#pops_mimic_cn = get_pops(newdata3, num, "cN_2local", "mimic")

In [196]:
pops_landscape_maximizes_c2 = get_pops(newdata4, num, "c2_2local", "landscape_maximizes")
pops_landscape_maximizes_c4 = get_pops(newdata4, num, "c4_2local", "landscape_maximizes")
#pops_landscape_maximizes_c5 = get_pops(newdata4, num, "c5_2local", "landscape_maximizes")
#pops_landscape_maximizes_c6 = get_pops(newdata4, num, "c6_2local", "landscape_maximizes")
#pops_landscape_maximizes_cn = get_pops(newdata4, num, "cN_2local", "landscape_maximizes")

In [197]:
pops_strongest_maximizes_c2 = get_pops(newdata5, num, "c2_2local", "strongest_maximizes")
pops_strongest_maximizes_c4 = get_pops(newdata5, num, "c4_2local", "strongest_maximizes")
#pops_strongest_maximizes_c5 = get_pops(newdata5, num, "c5_2local", "strongest_maximizes")
#pops_strongest_maximizes_c6 = get_pops(newdata5, num, "c6_2local", "strongest_maximizes")
#pops_strongest_maximizes_cn = get_pops(newdata5, num, "cN_2local", "strongest_maximizes")

In [198]:
pops_random_c2_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_random_c2)
pops_random_c4_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_random_c4)
#pops_random_c5_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_random_c5)
#pops_random_c6_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_random_c6)
#pops_random_cn_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_random_cn)

In [199]:
pops_greedy_c2_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_greedy_c2)
pops_greedy_c4_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_greedy_c4)
#pops_greedy_c5_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_greedy_c5)
#pops_greedy_c6_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_greedy_c6)
#pops_greedy_cn_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_greedy_cn)

In [200]:
pops_mimic_c2_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_mimic_c2)
pops_mimic_c4_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_mimic_c4)
#pops_mimic_c5_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_mimic_c5)
#pops_mimic_c6_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_mimic_c6)
#pops_mimic_cn_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_mimic_cn)

In [201]:
pops_landscape_maximizes_c2_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_landscape_maximizes_c2)
pops_landscape_maximizes_c4_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_landscape_maximizes_c4)
#pops_landscape_maximizes_c5_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_landscape_maximizes_c5)
#pops_landscape_maximizes_c6_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_landscape_maximizes_c6)
#pops_landscape_maximizes_cn_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_landscape_maximizes_cn)

In [202]:
pops_strongest_maximizes_c2_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_strongest_maximizes_c2)
pops_strongest_maximizes_c4_ens_avg = ensemble_averaged_one_point_measures_mean_std(pops_strongest_maximizes_c4)

In [203]:
#get all twoqdms for C2

In [204]:
size=12
twoQdmsrandomc2 = get_2_qbit_dms(newdata1,size,"c2_2local","random")
twoQdmsgreedyc2 = get_2_qbit_dms(newdata2,size,"c2_2local","greedy")
twoQdmsmimicc2 = get_2_qbit_dms(newdata3,size,"c2_2local","mimic")
twoQdmslandscape_maxc2 = get_2_qbit_dms(newdata4,size,"c2_2local","landscape_maximizes")
twoQdmsstrongest_maxc2 = get_2_qbit_dms(newdata5,size,"c2_2local","strongest_maximizes")

In [205]:
#get all twoqdms for C4

In [206]:
twoQdmsrandomc4 = get_2_qbit_dms(newdata1,size,"c4_2local","random")
twoQdmsgreedyc4 = get_2_qbit_dms(newdata2,size,"c4_2local","greedy")
twoQdmsmimicc4 = get_2_qbit_dms(newdata3,size,"c4_2local","mimic")
twoQdmslandscape_maxc4 = get_2_qbit_dms(newdata4,size,"c4_2local","landscape_maximizes")
twoQdmsstrongest_maxc4 = get_2_qbit_dms(newdata5,size,"c4_2local","strongest_maximizes")

In [158]:
#All extractable work

In [207]:
ext_work_all_random_c2 = np.array(extractable_work_of_all_trials(pops_random_c2))
ext_work_all_greedy_c2 = np.array(extractable_work_of_all_trials(pops_greedy_c2))
ext_work_all_mimic_c2 = np.array(extractable_work_of_all_trials(pops_mimic_c2))
ext_work_all_landscape_maximizes_c2 = np.array(extractable_work_of_all_trials(pops_landscape_maximizes_c2))
ext_work_all_strongest_maximizes_c2 = np.array(extractable_work_of_all_trials(pops_strongest_maximizes_c2))


/Users/unnatiakhouri/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:115: RuntimeWarning: divide by zero encountered in log
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)
/Users/unnatiakhouri/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:115: RuntimeWarning: invalid value encountered in scalar multiply
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)
/Users/unnatiakhouri/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:56: RuntimeWarning: invalid value encountered in log
  return 1 / (np.log((1 - pop) / pop))
/Users/unnatiakhouri/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:115: RuntimeWarning: invalid value encountered in log
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)
/Users/unnatiakhouri/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:64: RuntimeWarning: overflow encountered in exp
  pop = 1 / (1 + np.exp(1 / T))
/Users/unnatiakhouri/Documents/GitHub/Qnibble/Notebooks/.

In [208]:
ext_work_all_random_c4 = np.array(extractable_work_of_all_trials(pops_random_c4))
ext_work_all_greedy_c4 = np.array(extractable_work_of_all_trials(pops_greedy_c4))
ext_work_all_mimic_c4 = np.array(extractable_work_of_all_trials(pops_mimic_c4))
ext_work_all_landscape_maximizes_c4 = np.array(extractable_work_of_all_trials(pops_landscape_maximizes_c4))
ext_work_all_strongest_maximizes_c4 = np.array(extractable_work_of_all_trials(pops_strongest_maximizes_c4))

/Users/unnatiakhouri/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:116: RuntimeWarning: invalid value encountered in scalar multiply
  tr_2 = (1 - pop_1) * np.log(1 - pop_2) + (pop_1) * np.log(pop_2)


In [209]:
#Extractable work of the ensembled average trial per connectivity per rule W(<p>) where p is the landscape

In [210]:
ext_work_of_ens_avg_random_c2 = extractable_work_of_one_trial(pops_random_c2_ens_avg,0)
ext_work_of_ens_avg_random_c4 = extractable_work_of_one_trial(pops_random_c4_ens_avg,0)
#ext_work_of_ens_avg_random_c5 = extractable_work_of_one_trial(pops_random_c5_ens_avg,0)
##ext_work_of_ens_avg_random_c6 = extractable_work_of_one_trial(pops_random_c6_ens_avg,0)
#ext_work_of_ens_avg_random_cn = extractable_work_of_one_trial(pops_random_cn_ens_avg,0)

In [211]:
ext_work_of_ens_avg_greedy_c2 = extractable_work_of_one_trial(pops_random_c2_ens_avg,0)
ext_work_of_ens_avg_greedy_c4 = extractable_work_of_one_trial(pops_random_c4_ens_avg,0)
#ext_work_of_ens_avg_greedy_c5 = extractable_work_of_one_trial(pops_random_c5_ens_avg,0)
#ext_work_of_ens_avg_greedy_c6 = extractable_work_of_one_trial(pops_random_c6_ens_avg,0)
#ext_work_of_ens_avg_greedy_cn = extractable_work_of_one_trial(pops_random_cn_ens_avg,0)

In [212]:
ext_work_of_ens_avg_mimic_c2 = extractable_work_of_one_trial(pops_mimic_c2_ens_avg,0)
ext_work_of_ens_avg_mimic_c4 = extractable_work_of_one_trial(pops_mimic_c4_ens_avg,0)
#ext_work_of_ens_avg_mimic_c5 = extractable_work_of_one_trial(pops_mimic_c5_ens_avg,0)
#ext_work_of_ens_avg_mimic_c6 = extractable_work_of_one_trial(pops_mimic_c6_ens_avg,0)
#ext_work_of_ens_avg_mimic_cn = extractable_work_of_one_trial(pops_mimic_cn_ens_avg,0)

In [213]:
ext_work_of_ens_avg_landscape_maximizes_c2 = extractable_work_of_one_trial(pops_landscape_maximizes_c2_ens_avg,0)
ext_work_of_ens_avg_landscape_maximizes_c4 = extractable_work_of_one_trial(pops_landscape_maximizes_c4_ens_avg,0)
#ext_work_of_ens_avg_landscape_maximizes_c5 = extractable_work_of_one_trial(pops_landscape_maximizes_c5_ens_avg,0)
#ext_work_of_ens_avg_landscape_maximizes_c6 = extractable_work_of_one_trial(pops_landscape_maximizes_c6_ens_avg,0)
#ext_work_of_ens_avg_landscape_maximizes_cn = extractable_work_of_one_trial(pops_landscape_maximizes_cn_ens_avg,0)


In [214]:
ext_work_of_ens_avg_strongest_maximizes_c2 = extractable_work_of_one_trial(pops_strongest_maximizes_c2_ens_avg,0)
ext_work_of_ens_avg_strongest_maximizes_c4 = extractable_work_of_one_trial(pops_strongest_maximizes_c4_ens_avg,0)

In [215]:
#change in ext work all trials
change_in_ext_work_all_random_c2 = change_in_ext_work_all_trials(ext_work_all_random_c2)
change_in_ext_work_all_greedy_c2 = change_in_ext_work_all_trials(ext_work_all_greedy_c2)
change_in_ext_work_all_mimic_c2 = change_in_ext_work_all_trials(ext_work_all_mimic_c2)
change_in_ext_work_all_landscape_maximizes_c2 = change_in_ext_work_all_trials(ext_work_all_landscape_maximizes_c2)
change_in_ext_work_all_strongest_maximizes_c2 = change_in_ext_work_all_trials(ext_work_all_strongest_maximizes_c2)

/opt/homebrew/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:1496: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])


In [216]:
change_in_ext_work_all_random_c4 = change_in_ext_work_all_trials(ext_work_all_random_c4)
change_in_ext_work_all_greedy_c4 = change_in_ext_work_all_trials(ext_work_all_greedy_c4)
change_in_ext_work_all_mimic_c4 = change_in_ext_work_all_trials(ext_work_all_mimic_c4)
change_in_ext_work_all_landscape_maximizes_c4 = change_in_ext_work_all_trials(ext_work_all_landscape_maximizes_c4)
change_in_ext_work_all_strongest_maximizes_c4 = change_in_ext_work_all_trials(ext_work_all_strongest_maximizes_c4)

In [217]:
# compute and write out means and standard deviations for various statistics,
# where the mean std dev is computed over qubits,
# after each qubit is averaged over seeds, which are averaged over a ~50 step late-time window

In [218]:
# Mean and standard deviation for extractable work

In [219]:
#THIS DOES IT FOR CHANGE IN EXTRACTABLE WORK
W_stats_random_c2=late_time_averages_one_point_all_seeds_between_q(change_in_ext_work_all_random_c2)
W_stats_random_c4=late_time_averages_one_point_all_seeds_between_q(change_in_ext_work_all_random_c4)
W_stats_mimic_c2=late_time_averages_one_point_all_seeds_between_q(change_in_ext_work_all_mimic_c2)
W_stats_mimic_c4=late_time_averages_one_point_all_seeds_between_q(change_in_ext_work_all_mimic_c4)
W_stats_greedy_c2=late_time_averages_one_point_all_seeds_between_q(change_in_ext_work_all_greedy_c2)
W_stats_greedy_c4=late_time_averages_one_point_all_seeds_between_q(change_in_ext_work_all_greedy_c4)
W_stats_landscape_maximizes_c2=late_time_averages_one_point_all_seeds_between_q(change_in_ext_work_all_landscape_maximizes_c2)
W_stats_landscape_maximizes_c4=late_time_averages_one_point_all_seeds_between_q(change_in_ext_work_all_landscape_maximizes_c4)
W_stats_strongest_maximizes_c2=late_time_averages_one_point_all_seeds_between_q(change_in_ext_work_all_strongest_maximizes_c2)
W_stats_strongest_maximizes_c4=late_time_averages_one_point_all_seeds_between_q(change_in_ext_work_all_strongest_maximizes_c2)

/opt/homebrew/lib/python3.10/site-packages/numpy/_core/_methods.py:135: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


In [220]:
W_stats_greedy_c4

(np.float64(nan), np.float64(nan))

In [221]:
#THIS CODE DOES IT FOR EXTRACTABLE WORK
W_stats_random_c2=late_time_averages_one_point_all_seeds_between_q(ext_work_all_random_c2)
W_stats_random_c4=late_time_averages_one_point_all_seeds_between_q(ext_work_all_random_c4)
W_stats_mimic_c2=late_time_averages_one_point_all_seeds_between_q(ext_work_all_mimic_c2)
W_stats_mimic_c4=late_time_averages_one_point_all_seeds_between_q(ext_work_all_mimic_c4)
W_stats_greedy_c2=late_time_averages_one_point_all_seeds_between_q(ext_work_all_greedy_c2)
W_stats_greedy_c4=late_time_averages_one_point_all_seeds_between_q(ext_work_all_greedy_c4)
W_stats_landscape_maximizes_c2=late_time_averages_one_point_all_seeds_between_q(ext_work_all_landscape_maximizes_c2)
W_stats_landscape_maximizes_c4=late_time_averages_one_point_all_seeds_between_q(ext_work_all_landscape_maximizes_c4)
W_stats_strongest_maximizes_c2=late_time_averages_one_point_all_seeds_between_q(ext_work_all_strongest_maximizes_c2)
W_stats_strongest_maximizes_c4=late_time_averages_one_point_all_seeds_between_q(ext_work_all_strongest_maximizes_c2)

In [222]:
# Do the computations for Mutual Information

In [223]:
# Get the list of late-time-averaged (LTA) adjaceny matrices for all seeds

In [224]:
num_qubits=12
MI_adj_random_all_c2 = generate_all_adjacency_matrices(twoQdmsrandomc2, num_seeds, num_qubits, mutual_info_dicts)
MI_adj_greedy_all_c2 = generate_all_adjacency_matrices(twoQdmsgreedyc2, num_seeds, num_qubits, mutual_info_dicts)
MI_adj_mimic_all_c2 = generate_all_adjacency_matrices(twoQdmsmimicc2, num_seeds, num_qubits, mutual_info_dicts)
MI_adj_landscape_maximizes_all_c2 = generate_all_adjacency_matrices(twoQdmslandscape_maxc2, num_seeds, num_qubits, mutual_info_dicts)
MI_adj_strongest_maximizes_all_c2 = generate_all_adjacency_matrices(twoQdmsstrongest_maxc2, num_seeds, num_qubits, mutual_info_dicts)

/Users/unnatiakhouri/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:171: RuntimeWarning: divide by zero encountered in log
  from_eigen = -np.sum(eigen_vals * np.log(eigen_vals))
/Users/unnatiakhouri/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:171: RuntimeWarning: invalid value encountered in multiply
  from_eigen = -np.sum(eigen_vals * np.log(eigen_vals))
/Users/unnatiakhouri/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:182: RuntimeWarning: divide by zero encountered in log
  -d4 * np.log(d4) +
/Users/unnatiakhouri/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:182: RuntimeWarning: invalid value encountered in scalar multiply
  -d4 * np.log(d4) +
/Users/unnatiakhouri/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:183: RuntimeWarning: divide by zero encountered in log
  - 0.5 * (b2 + c3 - np.sqrt(b2 ** 2 + 4 * b3 * c2 - 2 * b2 * c3 + c3 ** 2)) * np.log(
/Users/unnatiakhouri/Documents/GitHub/Qnibble/Notebooks/../src/measurement

In [225]:
MI_adj_random_all_c4 = generate_all_adjacency_matrices(twoQdmsrandomc4, num_seeds, num_qubits, mutual_info_dicts)
MI_adj_greedy_all_c4 = generate_all_adjacency_matrices(twoQdmsgreedyc4, num_seeds, num_qubits, mutual_info_dicts)
MI_adj_mimic_all_c4 = generate_all_adjacency_matrices(twoQdmsmimicc4, num_seeds, num_qubits, mutual_info_dicts)
MI_adj_landscape_maximizes_all_c4 = generate_all_adjacency_matrices(twoQdmslandscape_maxc4, num_seeds, num_qubits, mutual_info_dicts)
MI_adj_strongest_maximizes_all_c4 = generate_all_adjacency_matrices(twoQdmsstrongest_maxc4, num_seeds, num_qubits, mutual_info_dicts)

In [226]:
LTA_adj_random_c2=late_time_averages_adj(MI_adj_random_all_c2)
LTA_adj_random_c4=late_time_averages_adj(MI_adj_random_all_c4)
LTA_adj_greedy_c2=late_time_averages_adj(MI_adj_greedy_all_c2)
LTA_adj_greedy_c4=late_time_averages_adj(MI_adj_greedy_all_c4)
LTA_adj_mimic_c2=late_time_averages_adj(MI_adj_mimic_all_c2)
LTA_adj_mimic_c4=late_time_averages_adj(MI_adj_mimic_all_c4)
LTA_adj_landscape_maximizes_c2=late_time_averages_adj(MI_adj_landscape_maximizes_all_c2)
LTA_adj_landscape_maximizes_c4=late_time_averages_adj(MI_adj_landscape_maximizes_all_c4)
LTA_adj_strongest_maximizes_c2=late_time_averages_adj(MI_adj_strongest_maximizes_all_c2)
LTA_adj_strongest_maximizes_c4=late_time_averages_adj(MI_adj_strongest_maximizes_all_c4)

In [82]:
#MI_total

In [227]:
MI_random_c2=np.sum(MItotal(LTA_adj_random_c2),axis=1)
MI_random_c4=np.sum(MItotal(LTA_adj_random_c4),axis=1)
MI_greedy_c2=np.sum(MItotal(LTA_adj_greedy_c2),axis=1)
MI_greedy_c4=np.sum(MItotal(LTA_adj_greedy_c4),axis=1)
MI_mimic_c2=np.sum(MItotal(LTA_adj_mimic_c2),axis=1)
MI_mimic_c4=np.sum(MItotal(LTA_adj_mimic_c4),axis=1)
MI_landscape_maximizes_c2=np.sum(MItotal(LTA_adj_landscape_maximizes_c2),axis=1)
MI_landscape_maximizes_c4=np.sum(MItotal(LTA_adj_landscape_maximizes_c4),axis=1)
MI_strongest_maximizes_c2=np.sum(MItotal(LTA_adj_strongest_maximizes_c2),axis=1)
MI_strongest_maximizes_c4=np.sum(MItotal(LTA_adj_strongest_maximizes_c4),axis=1)

In [228]:
MI_stats_random_c2=(np.mean(MI_random_c2),np.std(MI_random_c2))
MI_stats_random_c4=(np.mean(MI_random_c4),np.std(MI_random_c4))
MI_stats_greedy_c2=(np.mean(MI_greedy_c2),np.std(MI_greedy_c2))
MI_stats_greedy_c4=(np.mean(MI_greedy_c4),np.std(MI_greedy_c4))
MI_stats_mimic_c2=(np.mean(MI_mimic_c2),np.std(MI_mimic_c2))
MI_stats_mimic_c4=(np.mean(MI_mimic_c4),np.std(MI_mimic_c4))
MI_stats_landscape_maximizes_c2=(np.mean(MI_landscape_maximizes_c2),np.std(MI_landscape_maximizes_c2))
MI_stats_landscape_maximizes_c4=(np.mean(MI_landscape_maximizes_c4),np.std(MI_landscape_maximizes_c4))
MI_stats_strongest_maximizes_c2=(np.mean(MI_strongest_maximizes_c2),np.std(MI_strongest_maximizes_c2))
MI_stats_strongest_maximizes_c4=(np.mean(MI_strongest_maximizes_c4),np.std(MI_strongest_maximizes_c4))

In [229]:
# Clustering

In [230]:
clust_random_c2=clustering_coeff(LTA_adj_random_c2)
clust_random_c4=clustering_coeff(LTA_adj_random_c4)
clust_greedy_c2=clustering_coeff(LTA_adj_greedy_c2)
clust_greedy_c4=clustering_coeff(LTA_adj_greedy_c4)
clust_mimic_c2=clustering_coeff(LTA_adj_mimic_c2)
clust_mimic_c4=clustering_coeff(LTA_adj_mimic_c4)
clust_landscape_maximizes_c2=clustering_coeff(LTA_adj_landscape_maximizes_c2)
clust_landscape_maximizes_c4=clustering_coeff(LTA_adj_landscape_maximizes_c4)
clust_strongest_maximizes_c2=clustering_coeff(LTA_adj_strongest_maximizes_c2)
clust_strongest_maximizes_c4=clustering_coeff(LTA_adj_strongest_maximizes_c4)

In [231]:
clust_stats_random_c2=(np.mean(clust_random_c2),np.std(clust_random_c2))
clust_stats_random_c4=(np.mean(clust_random_c4),np.std(clust_random_c4))
clust_stats_greedy_c2=(np.mean(clust_greedy_c2),np.std(clust_greedy_c2))
clust_stats_greedy_c4=(np.mean(clust_greedy_c4),np.std(clust_greedy_c4))
clust_stats_mimic_c2=(np.mean(clust_mimic_c2),np.std(clust_mimic_c2))
clust_stats_mimic_c4=(np.mean(clust_mimic_c4),np.std(clust_mimic_c4))
clust_stats_landscape_maximizes_c2=(np.mean(clust_landscape_maximizes_c2),np.std(clust_landscape_maximizes_c2))
clust_stats_landscape_maximizes_c4=(np.mean(clust_landscape_maximizes_c4),np.std(clust_landscape_maximizes_c4))
clust_stats_strongest_maximizes_c2=(np.mean(clust_landscape_maximizes_c2),np.std(clust_strongest_maximizes_c2))
clust_stats_strongest_maximizes_c4=(np.mean(clust_landscape_maximizes_c4),np.std(clust_strongest_maximizes_c4))

In [232]:
# Disparity

In [233]:
disp_random_c2=disparity_function(LTA_adj_random_c2,num)
disp_random_c4=disparity_function(LTA_adj_random_c4,num)
disp_greedy_c2=disparity_function(LTA_adj_greedy_c2,num)
disp_greedy_c4=disparity_function(LTA_adj_greedy_c4,num)
disp_mimic_c2=disparity_function(LTA_adj_mimic_c2,num)
disp_mimic_c4=disparity_function(LTA_adj_mimic_c4,num)
disp_landscape_maximizes_c2=disparity_function(LTA_adj_landscape_maximizes_c2,num)
disp_landscape_maximizes_c4=disparity_function(LTA_adj_landscape_maximizes_c4,num)
disp_strongest_maximizes_c2=disparity_function(LTA_adj_strongest_maximizes_c2,num)
disp_strongest_maximizes_c4=disparity_function(LTA_adj_strongest_maximizes_c4,num)

In [234]:
disp_stats_random_c2=(np.mean(disp_random_c2),np.std(disp_random_c2))
disp_stats_random_c4=(np.mean(disp_random_c4),np.std(disp_random_c4))
disp_stats_greedy_c2=(np.mean(disp_greedy_c2),np.std(disp_greedy_c2))
disp_stats_greedy_c4=(np.mean(disp_greedy_c4),np.std(disp_greedy_c4))
disp_stats_mimic_c2=(np.mean(disp_mimic_c2),np.std(disp_mimic_c2))
disp_stats_mimic_c4=(np.mean(disp_mimic_c2),np.std(disp_mimic_c2))
disp_stats_landscape_maximizes_c2=(np.mean(disp_landscape_maximizes_c2),np.std(disp_landscape_maximizes_c2))
disp_stats_landscape_maximizes_c4=(np.mean(disp_landscape_maximizes_c4),np.std(disp_landscape_maximizes_c4))
disp_stats_strongest_maximizes_c2=(np.mean(disp_strongest_maximizes_c2),np.std(disp_strongest_maximizes_c2))
disp_stats_strongest_maximizes_c4=(np.mean(disp_strongest_maximizes_c4),np.std(disp_strongest_maximizes_c4))

In [235]:
# Dynamical map component tz

In [236]:
tzarray_random_c2=tz_full_array(pops_random_c2)
tzarray_random_c4=tz_full_array(pops_random_c4)
tzarray_greedy_c2=tz_full_array(pops_greedy_c2)
tzarray_greedy_c4=tz_full_array(pops_greedy_c4)
tzarray_mimic_c2=tz_full_array(pops_mimic_c2)
tzarray_mimic_c4=tz_full_array(pops_mimic_c4)
tzarray_landscape_maximizes_c2=tz_full_array(pops_landscape_maximizes_c2)
tzarray_landscape_maximizes_c4=tz_full_array(pops_landscape_maximizes_c4)
tzarray_strongest_maximizes_c2=tz_full_array(pops_strongest_maximizes_c2)
tzarray_strongest_maximizes_c4=tz_full_array(pops_strongest_maximizes_c4)

In [237]:
tz_stats_random_c2=late_time_averages_one_point_all_seeds_between_q(tzarray_random_c2)
tz_stats_random_c4=late_time_averages_one_point_all_seeds_between_q(tzarray_random_c4)
tz_stats_greedy_c2=late_time_averages_one_point_all_seeds_between_q(tzarray_greedy_c2)
tz_stats_greedy_c4=late_time_averages_one_point_all_seeds_between_q(tzarray_greedy_c4)
tz_stats_mimic_c2=late_time_averages_one_point_all_seeds_between_q(tzarray_mimic_c2)
tz_stats_mimic_c4=late_time_averages_one_point_all_seeds_between_q(tzarray_mimic_c4)
tz_stats_landscape_maximizes_c2=late_time_averages_one_point_all_seeds_between_q(tzarray_landscape_maximizes_c2)
tz_stats_landscape_maximizes_c4=late_time_averages_one_point_all_seeds_between_q(tzarray_landscape_maximizes_c4)
tz_stats_strongest_maximizes_c2=late_time_averages_one_point_all_seeds_between_q(tzarray_strongest_maximizes_c2)
tz_stats_strongest_maximizes_c4=late_time_averages_one_point_all_seeds_between_q(tzarray_strongest_maximizes_c4)

In [378]:
# Finally, write out all the stats

In [245]:
import csv

with open('RunStats.csv', mode='a') as csv_file:
    fieldnames = [
        'runtype_name', 'tz_mean', 'tz_std', 'MI_mean', 'MI_std', 'disp_mean', 'disp_std', 
        'clust_mean', 'clust_std', 'DeltaW_mean', 'DeltaW_std', 'DeltaW_persist_stat', 
        'DeltaW_persist_std', 'Persist_skew_mean', 'Persist_skew_std', 'Persist_kurtosis_mean', 
        'Persist_kurtosis_std'
    ]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Row 1
    writer.writerow({
        'runtype_name': 'R1Q8C2ICpure',
        'tz_mean': tz_stats_random_c2[0], 
        'tz_std': tz_stats_random_c2[1],
        'MI_mean': MI_stats_random_c2[0],
        'MI_std': MI_stats_random_c2[1], 
        'DeltaW_mean': W_stats_random_c2[0],
        'DeltaW_std': W_stats_random_c2[1],
        'disp_mean': disp_stats_random_c2[0],
        'disp_std': disp_stats_random_c2[1],
        'clust_mean': clust_stats_random_c2[0],
        'clust_std': clust_stats_random_c2[1],
        'DeltaW_persist_stat': Persistence_data_random_c2[0],
        'DeltaW_persist_std': Persistence_data_random_c2[1],
        'Persist_skew_mean': Persistence_data_random_c2[2],
        'Persist_skew_std': Persistence_data_random_c2[3],
        'Persist_kurtosis_mean': Persistence_data_random_c2[4],
        'Persist_kurtosis_std': Persistence_data_random_c2[5]
    })

    # Row 2
    writer.writerow({
        'runtype_name': 'R1Q8C4ICpure',
        'tz_mean': tz_stats_random_c4[0], 
        'tz_std': tz_stats_random_c4[1],
        'MI_mean': MI_stats_random_c4[0],
        'MI_std': MI_stats_random_c4[1], 
        'DeltaW_mean': W_stats_random_c4[0],
        'DeltaW_std': W_stats_random_c4[1],
        'disp_mean': disp_stats_random_c4[0],
        'disp_std': disp_stats_random_c4[1],
        'clust_mean': clust_stats_random_c4[0],
        'clust_std': clust_stats_random_c4[1],
        'DeltaW_persist_stat': Persistence_data_random_c4[0],
        'DeltaW_persist_std': Persistence_data_random_c4[1],
        'Persist_skew_mean': Persistence_data_random_c4[2],
        'Persist_skew_std': Persistence_data_random_c4[3],
        'Persist_kurtosis_mean': Persistence_data_random_c4[4],
        'Persist_kurtosis_std': Persistence_data_random_c4[5]
    })

    # Row 3
    writer.writerow({
        'runtype_name': 'R2Q8C2ICpure',
        'tz_mean': tz_stats_greedy_c2[0], 
        'tz_std': tz_stats_greedy_c2[1],
        'MI_mean': MI_stats_greedy_c2[0],
        'MI_std': MI_stats_greedy_c2[1], 
        'DeltaW_mean': W_stats_greedy_c2[0],
        'DeltaW_std': W_stats_greedy_c2[1],
        'disp_mean': disp_stats_greedy_c2[0],
        'disp_std': disp_stats_greedy_c2[1],
        'clust_mean': clust_stats_greedy_c2[0],
        'clust_std': clust_stats_greedy_c2[1],
        'DeltaW_persist_stat': Persistence_data_greedy_c2[0],
        'DeltaW_persist_std': Persistence_data_greedy_c2[1],
        'Persist_skew_mean': Persistence_data_greedy_c2[2],
        'Persist_skew_std': Persistence_data_greedy_c2[3],
        'Persist_kurtosis_mean': Persistence_data_greedy_c2[4],
        'Persist_kurtosis_std': Persistence_data_greedy_c2[5]
    })

    # Row 4
    writer.writerow({
        'runtype_name': 'R2Q8C4ICpure',
        'tz_mean': tz_stats_greedy_c4[0], 
        'tz_std': tz_stats_greedy_c4[1],
        'MI_mean': MI_stats_greedy_c4[0],
        'MI_std': MI_stats_greedy_c4[1], 
        'DeltaW_mean': W_stats_greedy_c4[0],
        'DeltaW_std': W_stats_greedy_c4[1],
        'disp_mean': disp_stats_greedy_c4[0],
        'disp_std': disp_stats_greedy_c4[1],
        'clust_mean': clust_stats_greedy_c4[0],
        'clust_std': clust_stats_greedy_c4[1],
        'DeltaW_persist_stat': Persistence_data_greedy_c4[0],
        'DeltaW_persist_std': Persistence_data_greedy_c4[1],
        'Persist_skew_mean': Persistence_data_greedy_c4[2],
        'Persist_skew_std': Persistence_data_greedy_c4[3],
        'Persist_kurtosis_mean': Persistence_data_greedy_c4[4],
        'Persist_kurtosis_std': Persistence_data_greedy_c4[5]
    })

    # Row 5
    writer.writerow({
        'runtype_name': 'R5Q8C2ICpure',
        'tz_mean': tz_stats_mimic_c2[0], 
        'tz_std': tz_stats_mimic_c2[1],
        'MI_mean': MI_stats_mimic_c2[0],
        'MI_std': MI_stats_mimic_c2[1], 
        'DeltaW_mean': W_stats_mimic_c2[0],
        'DeltaW_std': W_stats_mimic_c2[1],
        'disp_mean': disp_stats_mimic_c2[0],
        'disp_std': disp_stats_mimic_c2[1],
        'clust_mean': clust_stats_mimic_c2[0],
        'clust_std': clust_stats_mimic_c2[1],
        'DeltaW_persist_stat': Persistence_data_mimic_c2[0],
        'DeltaW_persist_std': Persistence_data_mimic_c2[1],
        'Persist_skew_mean': Persistence_data_mimic_c2[2],
        'Persist_skew_std': Persistence_data_mimic_c2[3],
        'Persist_kurtosis_mean': Persistence_data_mimic_c2[4],
        'Persist_kurtosis_std': Persistence_data_mimic_c2[5]
    })

    # Row 6
    writer.writerow({
        'runtype_name': 'R5Q8C4ICpure',
        'tz_mean': tz_stats_mimic_c4[0], 
        'tz_std': tz_stats_mimic_c4[1],
        'MI_mean': MI_stats_mimic_c4[0],
        'MI_std': MI_stats_mimic_c4[1], 
        'DeltaW_mean': W_stats_mimic_c4[0],
        'DeltaW_std': W_stats_mimic_c4[1],
        'disp_mean': disp_stats_mimic_c4[0],
        'disp_std': disp_stats_mimic_c4[1],
        'clust_mean': clust_stats_mimic_c4[0],
        'clust_std': clust_stats_mimic_c4[1],
        'DeltaW_persist_stat': Persistence_data_mimic_c4[0],
        'DeltaW_persist_std': Persistence_data_mimic_c4[1],
        'Persist_skew_mean': Persistence_data_mimic_c4[2],
        'Persist_skew_std': Persistence_data_mimic_c4[3],
        'Persist_kurtosis_mean': Persistence_data_mimic_c4[4],
        'Persist_kurtosis_std': Persistence_data_mimic_c4[5]
    })

    # Row 7
    writer.writerow({
        'runtype_name': 'R4Q8C2ICpure',
        'tz_mean': tz_stats_landscape_maximizes_c2[0], 
        'tz_std': tz_stats_landscape_maximizes_c2[1],
        'MI_mean': MI_stats_landscape_maximizes_c2[0],
        'MI_std': MI_stats_landscape_maximizes_c2[1], 
        'DeltaW_mean': W_stats_landscape_maximizes_c2[0],
        'DeltaW_std': W_stats_landscape_maximizes_c2[1],
        'disp_mean': disp_stats_landscape_maximizes_c2[0],
        'disp_std': disp_stats_landscape_maximizes_c2[1],
        'clust_mean': clust_stats_landscape_maximizes_c2[0],
        'clust_std': clust_stats_landscape_maximizes_c2[1],
        'DeltaW_persist_stat': Persistence_data_landscape_maximizes_c2[0],
        'DeltaW_persist_std': Persistence_data_landscape_maximizes_c2[1],
        'Persist_skew_mean': Persistence_data_landscape_maximizes_c2[2],
        'Persist_skew_std': Persistence_data_landscape_maximizes_c2[3],
        'Persist_kurtosis_mean': Persistence_data_landscape_maximizes_c2[4],
        'Persist_kurtosis_std': Persistence_data_landscape_maximizes_c2[5]
    })

    # Row 8
    writer.writerow({
        'runtype_name': 'R43Q8C4ICpure',
        'tz_mean': tz_stats_landscape_maximizes_c4[0], 
        'tz_std': tz_stats_landscape_maximizes_c4[1],
        'MI_mean': MI_stats_landscape_maximizes_c4[0],
        'MI_std': MI_stats_landscape_maximizes_c4[1], 
        'DeltaW_mean': W_stats_landscape_maximizes_c4[0],
        'DeltaW_std': W_stats_landscape_maximizes_c4[1],
        'disp_mean': disp_stats_landscape_maximizes_c4[0],
        'disp_std': disp_stats_landscape_maximizes_c4[1],
        'clust_mean': clust_stats_landscape_maximizes_c4[0],
        'clust_std': clust_stats_landscape_maximizes_c4[1],
        'DeltaW_persist_stat': Persistence_data_landscape_maximizes_c4[0],
        'DeltaW_persist_std': Persistence_data_landscape_maximizes_c4[1],
        'Persist_skew_mean': Persistence_data_landscape_maximizes_c4[2],
        'Persist_skew_std': Persistence_data_landscape_maximizes_c4[3],
        'Persist_kurtosis_mean': Persistence_data_landscape_maximizes_c4[4],
        'Persist_kurtosis_std': Persistence_data_landscape_maximizes_c4[5]
    })

    # Row 9
    writer.writerow({
        'runtype_name': 'R4Q8C2ICpure',
        'tz_mean': tz_stats_strongest_maximizes_c2[0], 
        'tz_std': tz_stats_strongest_maximizes_c2[1],
        'MI_mean': MI_stats_strongest_maximizes_c2[0],
        'MI_std': MI_stats_strongest_maximizes_c2[1], 
        'DeltaW_mean': W_stats_strongest_maximizes_c2[0],
        'DeltaW_std': W_stats_strongest_maximizes_c2[1],
        'disp_mean': disp_stats_strongest_maximizes_c2[0],
        'disp_std': disp_stats_strongest_maximizes_c2[1],
        'clust_mean': clust_stats_strongest_maximizes_c2[0],
        'clust_std': clust_stats_strongest_maximizes_c2[1],
        'DeltaW_persist_stat': Persistence_data_strongest_maximizes_c2[0],
        'DeltaW_persist_std': Persistence_data_strongest_maximizes_c2[1],
        'Persist_skew_mean': Persistence_data_strongest_maximizes_c2[2],
        'Persist_skew_std': Persistence_data_strongest_maximizes_c2[3],
        'Persist_kurtosis_mean': Persistence_data_strongest_maximizes_c2[4],
        'Persist_kurtosis_std': Persistence_data_strongest_maximizes_c2[5]
    })

    # Row 8
    writer.writerow({
        'runtype_name': 'R4Q8C4ICpure',
        'tz_mean': tz_stats_strongest_maximizes_c4[0], 
        'tz_std': tz_stats_strongest_maximizes_c4[1],
        'MI_mean': MI_stats_strongest_maximizes_c4[0],
        'MI_std': MI_stats_strongest_maximizes_c4[1], 
        'DeltaW_mean': W_stats_strongest_maximizes_c4[0],
        'DeltaW_std': W_stats_strongest_maximizes_c4[1],
        'disp_mean': disp_stats_strongest_maximizes_c4[0],
        'disp_std': disp_stats_strongest_maximizes_c4[1],
        'clust_mean': clust_stats_strongest_maximizes_c4[0],
        'clust_std': clust_stats_strongest_maximizes_c4[1],
        'DeltaW_persist_stat': Persistence_data_strongest_maximizes_c4[0],
        'DeltaW_persist_std': Persistence_data_strongest_maximizes_c4[1],
        'Persist_skew_mean': Persistence_data_strongest_maximizes_c4[2],
        'Persist_skew_std': Persistence_data_strongest_maximizes_c4[3],
        'Persist_kurtosis_mean': Persistence_data_strongest_maximizes_c4[4],
        'Persist_kurtosis_std': Persistence_data_strongest_maximizes_c4[5]
    })


In [160]:
W_stats_random_c2,W_stats_greedy_c2,W_stats_mimic_c2,W_stats_landscape_maximizes_c2,W_stats_strongest_maximizes_c2,W_stats_random_c4,W_stats_greedy_c4,W_stats_mimic_c4,W_stats_landscape_maximizes_c4,W_stats_strongest_maximizes_c4

((np.float64(0.0014011697553602187), np.float64(4.3401722831722103e-05)),
 (np.float64(0.004475818445240242), np.float64(0.0007833036478061397)),
 (np.float64(0.0018134548287057906), np.float64(0.000190355394984272)),
 (np.float64(0.004242795894987534), np.float64(0.0008599429337025168)),
 (np.float64(0.002985021731884964), np.float64(0.0013396129661015083)),
 (np.float64(0.000877969330291428), np.float64(4.3446485121254166e-05)),
 (np.float64(0.004909774525812672), np.float64(0.0003764849739382052)),
 (np.float64(0.0013190902399404028), np.float64(3.210707050964729e-05)),
 (np.float64(0.006392694225459604), np.float64(0.0017624881930161781)),
 (np.float64(0.002985021731884964), np.float64(0.0013396129661015083)))

In [164]:
#tz_stats_random_c2,tz_stats_random_c4,tz_stats_greedy_c2,tz_stats_greedy_c4,tz_stats_mimic_c2,tz_stats_mimic_c4,tz_stats_landscape_maximizes_c2,tz_stats_landscape_maximizes_c4,tz_stats_strongest_maximizes_c2,tz_stats_strongest_maximizes_c4
##clust_stats_random_c2,clust_stats_random_c4,clust_stats_greedy_c2,clust_stats_greedy_c4,clust_stats_mimic_c2,clust_stats_mimic_c4,clust_stats_landscape_maximizes_c2,clust_stats_landscape_maximizes_c4,clust_stats_strongest_maximizes_c2,clust_stats_strongest_maximizes_c4
#disp_stats_random_c2,disp_stats_random_c4,disp_stats_greedy_c2,disp_stats_greedy_c4,disp_stats_mimic_c2,disp_stats_mimic_c4,disp_stats_landscape_maximizes_c2,disp_stats_landscape_maximizes_c4,disp_stats_strongest_maximizes_c2,disp_stats_strongest_maximizes_c4
#MI_stats_random_c2,MI_stats_random_c4, MI_stats_greedy_c2,MI_stats_greedy_c4,MI_stats_mimic_c2,MI_stats_mimic_c4,MI_stats_landscape_maximizes_c2,MI_stats_landscape_maximizes_c4,MI_stats_strongest_maximizes_c2,MI_stats_strongest_maximizes_c4



((0.13229287824981822, 0.013819297029398154),
 (0.11113541448022397, 0.006276879518026151),
 (0.14018283469701778, 0.045317020969064876),
 (0.11727387112490129, 0.025748098250654397),
 (0.13197104507535948, 0.02798024016956893),
 (0.11135647224556103, 0.006296290802734502),
 (0.14968552870924745, 0.05687353861071746),
 (0.11866571970270688, 0.028834019433634315),
 (0.16668791985714185, 0.043816468755098606),
 (0.11463256589405055, 0.016698533404454288))

In [ ]:
#Under progress
#Functions for extractable work analysis 

In [67]:
#Functions for extractable work analysis 
#Gives a list where first element is the total number of positive steps and the second is a list of when/length of the positive change
def count_positive_steps(list_of_lists, index):
    total_positive_steps = 0
    consecutive_positive_steps = 0
    consecutive_lengths = []  # List to store lengths of consecutive positive sequences
    for lst in list_of_lists:
        if lst[index] > 0:
            total_positive_steps += 1
            consecutive_positive_steps += 1
        else:
            consecutive_lengths.append(consecutive_positive_steps)  # Record length of consecutive positive sequence
            consecutive_positive_steps = 0  # Reset consecutive count if value becomes non-positive
            
    # Add the last consecutive count if the list ends with a positive value
    if consecutive_positive_steps > 0:
        consecutive_lengths.append(consecutive_positive_steps)
        
    return total_positive_steps, consecutive_lengths

def tally_elements(lst):
    tally = {}
    for element in lst:
        tally[element] = tally.get(element, 0) + 1
    return tally

#Plots histogram using above tally dictionary
def plot_histogram(ax, tally, label):
    elements = list(tally.keys())
    counts = list(tally.values())

    ax.bar(elements, counts)
    ax.set_xlabel('Element')
    ax.set_ylabel('Frequency')
    ax.set_title(label)
    ax.legend(label)

#Get the total number of positive steps per qubit on the network as a list
def Num_steps_positive(change_in_ex_work):
    Num_steps_positive=[]
    for qubit_index in range(8):
        Num_steps_positive.append(count_positive_steps(change_in_ex_work,qubit_index)[0])
    return Num_steps_positive

#Get the list of when positive changes occur per qubit on the network as a list
def list_of_consecutive_positive_steps(change_in_ex_work):
    sum=[0]
    for qubit_id in range(8):
        sum = sum+count_positive_steps(change_in_ex_work,qubit_id)[1]
    return sum

In [68]:
#Gives the average length of interaval for which the qubit has positive change; does for each qubit and returns a list
def average_time_step_Q_positive(num_qubits,changein_Wex):
    average_time_positive = []
    for qubit_index in range(num_qubits):
        average_time_positive.append(np.mean(count_positive_steps(changein_Wex, qubit_index)[1]))
    return average_time_positive
    
#takes in a dictionary
def avg_time_step_positive(tally):
    num=0
    denom=0
    for key,values in tally.items():
        num = values*key+num
        denom=values+denom
    return num/denom

In [69]:
#get average of the cumulative positive change in extractable work since this is the value you can extract at any point from the landscape as resource
def replace_negatives_with_zero(lst):
    for sublist in lst:
        for i in range(len(sublist)):
            if sublist[i] < 0:
                sublist[i] = 0
    return lst

def replace_consecutive_positives_with_cumulative_sum(list_of_lists):
    lst = [list(column) for column in zip(*list_of_lists)]
    for sublist in lst:
        cumulative_sum = 0
        consecutive_positive_count = 0
        for i in range(len(sublist)):
            if sublist[i] > 0:
                cumulative_sum += sublist[i]
                consecutive_positive_count += 1
                if i == len(sublist) - 1 or sublist[i + 1] == 0:
                    if consecutive_positive_count > 1:
                        for j in range(i - consecutive_positive_count + 1, i + 1):
                            sublist[j] = 0
                        sublist[i] = cumulative_sum
                    cumulative_sum = 0
                    consecutive_positive_count = 0
            else:
                cumulative_sum = 0
                consecutive_positive_count = 0
    return lst

In [70]:
def list_of_consec_positive_steps_no_zeros(change_in):
    listt=[]
    for ele in list_of_consecutive_positive_steps(change_in):
        if ele != 0:
            listt.append(ele)
    return listt

def consecutive_positives_with_cumulative_sum_no_zeores(change_in):
    list2=[]
    for ele in np.array(replace_consecutive_positives_with_cumulative_sum(replace_negatives_with_zero(change_in))).flatten():
        if ele!=0:
            list2.append(ele)
    return list2

In [71]:
def average_ex_resource_landscape(change_in_ex_work):
    means=[]
    for qubit in range(8):
        means.append(np.mean(replace_consecutive_positives_with_cumulative_sum(replace_negatives_with_zero(change_in_ex_work))[qubit]))
    return np.mean(means)

def average_cumulative_Wex(num_qubits,changein_Wex):
    average_time_positive = []
    for qubit_index in range(num_qubits):
        average_time_positive.append(np.mean(count_positive_steps(changein_Wex, qubit_index)[1]))
    return average_time_positive
    
def total_wex(num_qubits,changein_Wex):
    total_wex = []
    for qubit_index in range(num_qubits):
        changein_Wex[0:99,qubit_index]
        total_wex.append(sum(changein_Wex[0:99,qubit_index]))
    return total_wex

In [99]:
def plot_histogram_with_tally(tally_random,tally_greedy,tally_mimic,tally_landmax,tally_smax):
    del tally_random[0]
    del tally_greedy[0]
    del tally_mimic[0]
    del tally_landmax[0]
    del tally_smax[0]
    
    fig, axs = plt.subplots(1, 5, figsize=(15, 5))
        
    plot_histogram(axs[0], tally_random, 'R1 ')
    plot_histogram(axs[1], tally_greedy, 'R2')
    plot_histogram(axs[2], tally_mimic, 'R3')
    plot_histogram(axs[3], tally_landmax, 'R4')
    plot_histogram(axs[4], tally_smax, 'R5')

    plt.savefig(f'histogram_freq_temp_var_{conn}_dataset_{label}_ens_avg_change_in_ext_work.png')

    plt.tight_layout()
    plt.show()

In [132]:
counts,bin_edges=np.histogram(list_of_consec_positive_steps_no_zeros(ens_avg_change_in_ext_work_random_c2),20)

NameError: name 'ens_avg_change_in_ext_work_random_c2' is not defined

In [72]:
#C2 Histogram
Num_steps_positive_random = Num_steps_positive(ens_avg_change_in_ext_work_random_c2)
Num_steps_positive_greedy = Num_steps_positive(ens_avg_change_in_ext_work_greedy_c2)
Num_steps_positive_mimic = Num_steps_positive(ens_avg_change_in_ext_work_mimic_c2)
Num_steps_positive_landmax = Num_steps_positive(ens_avg_change_in_ext_work_landscape_maximizes_c2)
Num_steps_positive_smax = Num_steps_positive(ens_avg_change_in_ext_work_strongest_maximizes_c2)

tally_random=tally_elements(list_of_consecutive_positive_steps(ens_avg_change_in_ext_work_random_c2))
tally_greedy=tally_elements(list_of_consecutive_positive_steps(ens_avg_change_in_ext_work_greedy_c2))
tally_landmax=tally_elements(list_of_consecutive_positive_steps(ens_avg_change_in_ext_work_landscape_maximizes_c2))
tally_mimic=tally_elements(list_of_consecutive_positive_steps(ens_avg_change_in_ext_work_mimic_c2))
tally_smax=tally_elements(list_of_consecutive_positive_steps(ens_avg_change_in_ext_work_strongest_maximizes_c2))

conn="c2"
plot_histogram_with_tally(tally_random,tally_greedy,tally_mimic,tally_landmax,tally_smax)

NameError: name 'ens_avg_change_in_ext_work_random_c2' is not defined

In [72]:
# Create a DataFrame from the histogram data
hist_df = pd.DataFrame({
    'bin_left_edge': bin_edges[:-1],  # Left edge of each bin
    'bin_right_edge': bin_edges[1:],  # Right edge of each bin
    'count': counts                   # Frequency of data in each bin
})
# Print the DataFrame
print(hist_df)

NameError: name 'bin_edges' is not defined

In [73]:
#Histogram stats:
#Here we will compute, meadian, mean, mode, skewness, modality, kurtosis
#Some additional terms are interval time frequency for each histogram
def positive_interval_data(data):
    #Assuming data list is list_of_consec_positive_steps_no_zeros(change_in_ext_work_ens_avg_random_c2)
    #this will be for one seed
    tally_dict = tally_elements(data)
    min_val = int(np.floor(np.min(data)))  # Round down to nearest integer
    max_val = int(np.ceil(np.max(data)))
    bins = np.arange(min_val, max_val + 2)
    counts=list(tally_dict.values())
    bin_edges=categories = list(tally_dict.keys())

    mean = np.mean(data)
    median = np.median(data)
    mode = categories[np.argmax(counts)]
    std_dev = np.std(data)
    data_skewness = skew(data)
    data_kurtosis = kurtosis(data)
    peak_interval = categories[np.argmax(counts)]*np.max(counts)

    # Analyzing the shape of the histogram
    # Symmetry check (comparing mean and median for skewness insight)
    is_symmetric = np.isclose(mean, median)

    return (mean,median,mode,std_dev,data_skewness,data_kurtosis,peak_interval)
    

In [74]:
def positive_interval_data_all_seed(list_of_lists):
    stats_each_trial=[]
    for l in list_of_lists:
        stats_each_trial.append(positive_interval_data(l))
    stats_each_trial=np.array(stats_each_trial)
    peak_interval_all_trial=stats_each_trial[:,6]
    mean_all_trial=stats_each_trial[:,0]
    kurtosis_all_trials = stats_each_trial[:,5]
    skewness_all_trials = stats_each_trial[:,4]
    mean_peak = np.mean(peak_interval_all_trial,axis=0)
    std_peak =np.std(peak_interval_all_trial,axis=0)
    mean_kurtosis = np.mean(kurtosis_all_trials)
    std_kurtosis = np.std(kurtosis_all_trials)
    mean_skew = np.mean(skewness_all_trials)
    std_skew = np.std(skewness_all_trials)
    #similarly do for any other stats you want
    return [mean_peak,std_peak,mean_skew,std_skew,mean_kurtosis,std_kurtosis]

In [75]:
# list of consecutive positive for all trials

In [238]:
list_of_lists_consecutive_random_c2=[]
for change_in_ext_work_one_trial in change_in_ext_work_all_random_c2:
    list_of_lists_consecutive_random_c2.append(list_of_consec_positive_steps_no_zeros(change_in_ext_work_one_trial))

list_of_lists_consecutive_greedy_c2=[]
for change_in_ext_work_one_trial in change_in_ext_work_all_greedy_c2:
    list_of_lists_consecutive_greedy_c2.append(list_of_consec_positive_steps_no_zeros(change_in_ext_work_one_trial))

list_of_lists_consecutive_mimic_c2=[]
for change_in_ext_work_one_trial in change_in_ext_work_all_mimic_c2:
    list_of_lists_consecutive_mimic_c2.append(list_of_consec_positive_steps_no_zeros(change_in_ext_work_one_trial))

list_of_lists_consecutive_landscape_maximizes_c2=[]
for change_in_ext_work_one_trial in change_in_ext_work_all_landscape_maximizes_c2:
    list_of_lists_consecutive_landscape_maximizes_c2.append(list_of_consec_positive_steps_no_zeros(change_in_ext_work_one_trial))

list_of_lists_consecutive_strongest_maximizes_c2=[]
for change_in_ext_work_one_trial in change_in_ext_work_all_strongest_maximizes_c2:
    list_of_lists_consecutive_strongest_maximizes_c2.append(list_of_consec_positive_steps_no_zeros(change_in_ext_work_one_trial))


In [239]:
list_of_lists_consecutive_random_c4=[]
for change_in_ext_work_one_trial in change_in_ext_work_all_random_c4:
    list_of_lists_consecutive_random_c4.append(list_of_consec_positive_steps_no_zeros(change_in_ext_work_one_trial))

list_of_lists_consecutive_greedy_c4=[]
for change_in_ext_work_one_trial in change_in_ext_work_all_greedy_c4:
    list_of_lists_consecutive_greedy_c4.append(list_of_consec_positive_steps_no_zeros(change_in_ext_work_one_trial))

list_of_lists_consecutive_mimic_c4=[]
for change_in_ext_work_one_trial in change_in_ext_work_all_mimic_c4:
    list_of_lists_consecutive_mimic_c4.append(list_of_consec_positive_steps_no_zeros(change_in_ext_work_one_trial))

list_of_lists_consecutive_landscape_maximizes_c4=[]
for change_in_ext_work_one_trial in change_in_ext_work_all_landscape_maximizes_c4:
    list_of_lists_consecutive_landscape_maximizes_c4.append(list_of_consec_positive_steps_no_zeros(change_in_ext_work_one_trial))

list_of_lists_consecutive_strongest_maximizes_c4=[]
for change_in_ext_work_one_trial in change_in_ext_work_all_strongest_maximizes_c4:
    list_of_lists_consecutive_strongest_maximizes_c4.append(list_of_consec_positive_steps_no_zeros(change_in_ext_work_one_trial))

In [171]:
#tally_elements(list_of_lists_consecutive_random[1])
c=list(tally_elements(list_of_lists_consecutive_random[15]).values())
cas = list(tally_elements(list_of_lists_consecutive_random[15]).keys())
[c,cas]
positive_interval_data(list_of_lists_consecutive_random[15])

NameError: name 'list_of_lists_consecutive_random' is not defined

In [240]:
#mean_peak_length, std_peak_length, mean_skew, std_skew, mean_kurtosis, std_kurtosis
Persistence_data_random_c2=positive_interval_data_all_seed(list_of_lists_consecutive_random_c2)
Persistence_data_random_c4=positive_interval_data_all_seed(list_of_lists_consecutive_random_c4)


In [241]:
Persistence_data_landscape_maximizes_c2=positive_interval_data_all_seed(list_of_lists_consecutive_landscape_maximizes_c2)
Persistence_data_landscape_maximizes_c4=positive_interval_data_all_seed(list_of_lists_consecutive_landscape_maximizes_c4)
Persistence_data_landscape_maximizes_c2

[np.float64(532.79),
 np.float64(217.99854563735053),
 np.float64(0.3774736562103202),
 np.float64(0.47383678894979553),
 np.float64(-0.29571586879291856),
 np.float64(1.0010322022905465)]

In [242]:
Persistence_data_greedy_c2=positive_interval_data_all_seed(list_of_lists_consecutive_greedy_c2)
Persistence_data_greedy_c4=positive_interval_data_all_seed(list_of_lists_consecutive_greedy_c4)

In [243]:
Persistence_data_mimic_c2=positive_interval_data_all_seed(list_of_lists_consecutive_mimic_c2)
Persistence_data_mimic_c4=positive_interval_data_all_seed(list_of_lists_consecutive_mimic_c4)

In [244]:
Persistence_data_strongest_maximizes_c2=positive_interval_data_all_seed(list_of_lists_consecutive_strongest_maximizes_c2)
Persistence_data_strongest_maximizes_c4=positive_interval_data_all_seed(list_of_lists_consecutive_strongest_maximizes_c4)